<table width=100%>
    <tr>
        <td width = 40% align = "left">
            <h3> MTH786 Machine Learning with Python</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Semester A </h3>
        </td>
    </tr>
    <tr>
        <td width = 40% align = "left">
            <h3> Coursework 8</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Dr Nicola Perra </h3>
        </td>
    </tr>
</table>

In [ ]:
import numpy as np
from numpy.testing import assert_array_almost_equal, assert_array_equal
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

### Nearest neighbour classification
In this exercise you will work with a data set you are familiar with: $\mathtt{heigh\_weight.csv}$. By completing this exercise you will learn how to solve binary classification problem with non-parametric method. In the second part we will learn how to solve the classification problem for titanic data using logistic regression method.

We start by loading data. **Important:** please check that file $\mathtt{height\_weight.csv}$ is located in the same folder with your Jupyter notebook.

In [ ]:
types = np.genfromtxt("height_weight.csv", delimiter=",", skip_header=1, usecols=[0], \
                        converters={0: lambda x: 0 if b"A" in x else 1})
heights = np.genfromtxt("height_weight.csv",
                        delimiter=",",
                        skip_header=1,
                        usecols=[1])
weights = np.genfromtxt("height_weight.csv",
                        delimiter=",",
                        skip_header=1,
                        usecols=[2])
indexes = np.argsort(heights)

height_weight_input = np.c_[heights[indexes], weights[indexes]]
height_weight_output = types[indexes]


def standardise(data_matrix):
    row_of_means = np.mean(data_matrix, axis=0)
    standardised_matrix = data_matrix - row_of_means
    row_of_stds = np.std(standardised_matrix, axis=0)
    return (standardised_matrix / row_of_stds), row_of_means, row_of_stds


def de_standardise(standardised_matrix, row_of_means, row_of_stds):
    matrix = np.copy(standardised_matrix * row_of_stds)
    return matrix + row_of_means


height_weight_input, height_weight_row_of_means, height_weight_row_of_stds = standardise(
    height_weight_input)


def plot_height_weight(data_input, data_output):
    height_weight_de_standardised = de_standardise(data_input,
                                                   height_weight_row_of_means,
                                                   height_weight_row_of_stds)
    plt.scatter(height_weight_de_standardised[:,0][data_output == 0], \
                height_weight_de_standardised[:,1][data_output == 0], s=1, c = 'r')
    plt.scatter(height_weight_de_standardised[:,0][data_output == 1], \
                height_weight_de_standardised[:,1][data_output == 1], s=1, c = 'b')
    plt.xlabel('Height', fontsize=16)
    plt.xticks(fontsize=16)
    plt.ylabel('Weight', fontsize=16)
    plt.yticks(fontsize=16)
    plt.tight_layout
    plt.show()

In [ ]:
plot_height_weight(height_weight_input, height_weight_output)

Our main goal is to identify the type by studying their height and weight. We will end up with building a classifier which will split the height-weight plane into two regions: one corresponding to type A and another to type B. To achieve this we suggest to use the $K$-nearest neighbour classification method.

1. Write the function $\mathtt{pairwise\_distances}$  that takes two arguments:  $\mathtt{from\_data}$, $\mathtt{to\_data}$ - both are NumPy arrays of the same width that may have different height. These arrays represent two collections (later testing data and training data) of row-vectors, each describing a data point in  $d$ -dimensional input space. In other words for $$ 
\mathtt{from\_data} =
\begin{pmatrix}
x^{(1)}_1 & x^{(1)}_2 & \ldots & x^{(1)}_d \\
x^{(2)}_1 & x^{(2)}_2 & \ldots & x^{(2)}_d \\
\vdots & \vdots & \ddots & \vdots \\
x^{(p)}_1 & x^{(p)}_2 & \ldots & x^{(p)}_d 
\end{pmatrix}
, 
\quad
\mathtt{to\_data} =
\begin{pmatrix}
y^{(1)}_1 & y^{(1)}_2 & \ldots & y^{(1)}_d \\
y^{(2)}_1 & y^{(2)}_2 & \ldots & y^{(2)}_d \\
\vdots & \vdots & \ddots & \vdots \\
y^{(q)}_1 & y^{(q)}_2 & \ldots & y^{(q)}_d 
\end{pmatrix}
$$
your function should return a NumPy array of size $p\times q$ of the form
$$
\mathtt{pairwise\_distances} = 
\begin{pmatrix}
\mathrm{dist}\left(\mathbf{x^{(1)}},\mathbf{y^{(1)}}\right) &
\mathrm{dist}\left(\mathbf{x^{(1)},y^{(2)}}\right) & \ldots & \mathrm{dist}\left(\mathbf{x^{(1)},y^{(q)}}\right) \\
\mathrm{dist}\left(\mathbf{x^{(2)},y^{(1)}}\right) &
\mathrm{dist}\left(\mathbf{x^{(2)},y^{(2)}}\right) & \ldots & \mathrm{dist}\left(\mathbf{x^{(2)},y^{(q)}}\right) \\
\vdots &
\vdots & \ddots & \vdots\\
\mathrm{dist}\left(\mathbf{x^{(p)},y^{(1)}}\right) &
\mathrm{dist}\left(\mathbf{x^{(p)},y^{(2)}}\right) & \ldots & \mathrm{dist}\left(\mathbf{x^{(p)},y^{(q)}}\right)
\end{pmatrix}.
$$

In [ ]:
# the pairwise distance is computed considering all the rows of from-data against all the rows of y
def pairwise_distances(from_data, to_data):
    return np.sqrt(np.sum((from_data[:, np.newaxis, :] - to_data[np.newaxis, :, :])**2,axis=2))
# note how np.newaxis is used to extend the order of the array

In [ ]:
# this is a more explicit alternative of the same code
def pairwise_distances_2(from_data, to_data):
    distance=np.zeros((len(from_data),len(to_data)),float)
    counter_1=0
    for vector_x in from_data:
        first_vector=vector_x
        counter_2=0
        for vector_y in to_data:
            second_vector=vector_y
            distance[counter_1,counter_2]=np.sqrt(np.sum((vector_x-vector_y)**2))
            counter_2+=1
        counter_1+=1 
    return distance

Test your function with the following unit tests

In [ ]:
test_from_data = np.array([[0, 1, 2], [3, 4, 5]])
test_to_data = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]])
assert_array_almost_equal(pairwise_distances(test_from_data, test_to_data), np.array([[ 0., 5.19615242, 10.39230485, 15.58845727],\
                                                                                      [ 5.19615242,  0., 5.19615242, 10.39230485]]))

2. Write a function $\mathtt{nearest\_neighbour\_classification}$ that performs nearest neighbour classiffication as introduced in the lecture notes and that takes four arguments 
- $\mathtt{testing\_inputs}$ - a NumPy array of input samples to be classified
- $\mathtt{training\_inputs}$ - a NumPy array of data points for which the classification is known
- $\mathtt{training\_outputs}$ - a NumPy array of class labels for $\mathtt{training\_inputs}$ data
- $\mathtt{no\_of\_neighbours}$ - an integer argument that specifies how many neighbours we take into account to determine the class labels for the $\mathtt{testing\_inputs}$. Use the Euclidean norm to measure the distance between samples.

To implement the classification algorithm you may follow the below steps:  

a. Calculate the pairwise distances between testing and training data

b. Sort the pairwise distances by rows

c. For every row (data input) evaluate labels of the closest $K$ points 

d. Assign a data input with the class label that appears the most in between $K$ closest points

## Example
To better understand how the general code works, let us consider a simple example where

$\mathtt{data\_inputs} =
\begin{pmatrix}
15\\
6 \\
18
\end{pmatrix}$
these are the points we would like to classify. We do that considering the training data for which we know the correct labels. 

$\mathtt{training\_inputs} =
\begin{pmatrix}
10\\
8 \\
17\\
5\\
12\\
10\\
20\\
1\\
11
\end{pmatrix}$

The pairwise distance matrix will then be a 3x9 matrix. The first entry will be $\sqrt{(15-10)^2}=5$. 
Let's see:

In [ ]:
data_inputs = np.array([[15], [6], [18]])

training_inputs = np.array([[10], [8], [17], [5], [12], [10], [20], [1],[11]])


distances = pairwise_distances(data_inputs, training_inputs)
print (distances)

Let's now assume that the classification/labels for the 9 data points in the training set is

$\mathtt{training\_outputs} =
\begin{pmatrix}
1\\
1 \\
1\\
0\\
0\\
1\\
1\\
0\\
1
\end{pmatrix}$

We have two classes in this case. This is an example of binary classification

In [ ]:
training_outputs = np.array([1, 1, 1, 0, 0, 1, 1, 0, 1])
    
no_of_classes = 1 + np.max(training_outputs) # since the max is 1 and we have 0 or 1, this is the number of classes

# we can use this line to get the indices of the sorted distances
sorted_indices = np.argsort(distances, axis=1)
# let's check the distances
print (distances)
# and let's check the indices of the sorted values
print (sorted_indices)
# see how in the first row the value 2 is the smallest this corresponds to index=2 etc..

In [ ]:
# to simplify drastically the code we can make use of some numpy functionalities
# Consider the following example

a=np.array([3,4,5])

np.broadcast_to(a,(4,3)) # this function takes an array and creates another with different dimensions replicating it

In [ ]:
# also consider an array
a = np.array([[10, 30, 20], [60, 40, 50]])

# we can use argsort to get the indices of the sorted values
ai = np.argsort(a, axis=1)
print (ai) # let's check the output

# than we can use take_along_axis to get the array sorted! Basically we take the values of a but ordered following ai
np.take_along_axis(a, ai, axis=1)

In [ ]:
# we can use these two functions to get the labels of the closest training value for each data input!

no_of_points= len(training_inputs)
no_of_inputs = len(data_inputs)

# we can use broadcast_to to create, for each data_input, a row with all the training outputs
new_array_to_sort = np.broadcast_to(training_outputs,(no_of_inputs, no_of_points))
print (new_array_to_sort)
print ('----')

# now we can sort them using their distances
sorted_labels=np.take_along_axis(new_array_to_sort,sorted_indices, 1)
print (sorted_labels)

This means that for the data point 15, which we would like to classify, the closest point has label 1, the second closest label 0, all the other closest, but for the last two (which are the one more distant) have label 1. 

Now we are done, because we can classify a data input as the most common class considering the first k neighbors!
So we can see the classification already looking at the matrix above. If we use k=1, the first data point (15) will be 1, the second (6) will be 0 and the third (8) will be 1. 

How can we do this more in general? 

In [ ]:
# we need to speficy the number of K to consider
no_of_neighbours=1


# we can do the classification!
predicted_labels=np.zeros(no_of_inputs,int)
for id_input in range(no_of_inputs): # for each input we compute the average of label of the k closest
    probability_labels=np.zeros(no_of_classes,float)
    # this is an array that stores the probabilty that each input is member of a particular class
    # by looking that the fraction of k neighbors that are in the class
    for id_neighbor in range(0,no_of_neighbours):
        label=sorted_labels[id_input][id_neighbor] # this is the label of the neighbors
        probability_labels[label]+=1./no_of_neighbours

    predicted_labels[id_input]=np.argmax(probability_labels)
    
print (predicted_labels)

In [ ]:
# we can put all together!
def nearest_neighbour_classification(testing_inputs, training_inputs,training_outputs, no_of_neighbours):
    
    # first thing is to compute pairwise distances
    distances = pairwise_distances(testing_inputs, training_inputs)
    
    no_of_classes = 1 + np.max(training_outputs)
    sorted_indices = np.argsort(distances, axis=1)
    
    no_of_inputs = len(testing_inputs) # these are the data that we are trying to classify
    no_of_points = len(training_inputs)
    
    new_array_to_sort = np.broadcast_to(training_outputs,(no_of_inputs, no_of_points))
    sorted_labels = np.take_along_axis(new_array_to_sort,sorted_indices, 1)
    
    predicted_labels=np.zeros(no_of_inputs,int)
    for id_input in range(no_of_inputs):
        
        probability_labels=np.zeros(no_of_classes,float)
        
        for id_neighbor in range(0,no_of_neighbours):
            probability_labels[sorted_labels[id_input][id_neighbor]]+=1./no_of_neighbours

        predicted_labels[id_input]=np.argmax(probability_labels)
    
    
    return predicted_labels

Test your function with the following unit tests

In [ ]:
test_data_inputs = np.array([[15], [6], [18]])
test_training_inputs = np.array([[10], [8], [17], [5], [12], [10], [20], [1],
                                 [11]])
test_training_outputs = np.array([1, 1, 1, 0, 0, 1, 1, 0, 1])
test_no_of_neighbours = 1
nearest_neighbour_classification(test_data_inputs, test_training_inputs,test_training_outputs, test_no_of_neighbours)

In [ ]:
# it works also with multiple classes
test_data_inputs = np.array([[16, 0], [13, 6], [6, 5], [14, 15]])
test_training_inputs = np.array([[ 7, 13], [ 2,  4], [ 8, 15], [16, 19], [18,  3], [11, 19], [13,  4], [ 7, 18], [ 4, 14],\
                                 [10,  6], [17, 19], [19,  7], [ 4, 19], [18, 11], [18,  3], [14, 15], [14,  0], [ 5, 17],\
                                 [ 5, 15], [ 7, 19]])
test_training_outputs = np.array(
    [0, 2, 1, 0, 1, 0, 1, 1, 2, 1, 2, 2, 1, 1, 0, 1, 0, 0, 1, 2])
test_no_of_neighbours = 3
assert_array_almost_equal(nearest_neighbour_classification(test_data_inputs, test_training_inputs, \
                                                           test_training_outputs, test_no_of_neighbours),\
                          np.array([0, 1, 1, 0]))

3. Write a function $\mathtt{classification\_accuracy}$ that takes two NumPy array arguments $\mathtt{predicted\_labels}$ and $\mathtt{true\_labels}$ and evaluates the ratio of labels that coincide in both arrays to the number of all labels.

In [ ]:
def classification_accuracy(estimated_labels, true_labels):
    equal_labels = estimated_labels == true_labels # this returns an array made of true and false 
    return np.mean(equal_labels) # the average is how many true we have

Test your function with the following unit tests

In [ ]:
test_estimated_labels = np.array([0,4,0,0,2,4,0,0,2,2,2,3,3,1,3,4,0,3,4,0,1,1,2,0,0,0,\
                                  1,4,4,4,3,0,4,2,4,4,4,2,2,1,4,3,2,2,1,1,4,3,3,0,4,3,\
                                  0,0,0,2,4,3,4,3,1,3,2,4,2,3,2,3,2,3,1,0,4,3,2,3,1,3,\
                                  4,1,3,1,4,0,4,4,1,2,3,1,1,4,3,1,3,0,2,0,0,1])
test_true_labels = np.array([4,4,4,4,1,3,0,4,1,0,2,3,1,2,0,1,3,4,3,4,4,4,2,0,3,4,3,2,3,\
                             0,1,0,4,3,2,2,1,2,2,1,4,0,4,4,3,0,0,0,3,2,3,0,0,3,3,4,2,2,\
                             3,4,2,3,3,0,0,2,0,3,0,4,4,4,4,1,2,3,1,3,1,1,2,0,2,3,3,3,4,\
                             0,0,3,1,0,3,0,3,1,0,0,4,4])

assert_array_almost_equal(
    classification_accuracy(test_estimated_labels, test_true_labels), 0.26)

4. After we have learned how to classify the data, we would like to learn how one can optimise a value $K$ of neighbours participating in the classification. This would be done by performing standard $K$-fold cross validation. Below we provide you with some functions discussed in previous assignments. These functions are used to perform $K$-fold cross validation and consecutive grid search for hyperparameter optimisation.

In [ ]:
def KFold_split(data_size, K):
    np.random.seed(123456789)
    indexes = np.random.permutation(data_size)
    m, r = divmod(data_size, K)
    indexes_split = [
        indexes[i * m + min(i, r):(i + 1) * m + min(i + 1, r)]
        for i in range(K)
    ]
    return indexes_split


def KFold_cross_validation_knn(data_inputs, data_outputs, K, labels_evaluation,
                           missclassification_evaluation,knn):
    
    data_size = len(data_inputs)
    indexes_split = KFold_split(data_size, K)

    average_accuracy = 0
    for i in range(K):
        training_indexes = np.concatenate([indexes_split[j] for j in range(K) if (j != i)])
        
        predicted_labels = labels_evaluation(data_inputs[indexes_split[i]],
                                             data_inputs[training_indexes],
                                             data_outputs[training_indexes],knn)
        
        accuracy = missclassification_evaluation(predicted_labels,
                                              data_outputs[indexes_split[i]])
        average_accuracy += accuracy / K
        
    error = 1.-average_accuracy
    return error

5. We will also need a grid search function being implemented to find an optimal value of a number of neighbours to be included in the classification algorithm.

In [ ]:
def grid_search(objective, grid):
    values = np.array([])
    for point in grid:
        values = np.append(values, objective(point))
    return grid[np.argmin(values)]

6. Now define a grid of $K$ (number of neighbours to be included in the classification) values to be $\left[1,2,\ldots,20\right]$ and determine an optimal parameter $\mathtt{no\_of\_neighbours}$ based on K-fold cross validation and grid search. Assign a variable $\mathtt{optimal\_no\_of\_neighbours}$ with your result. For $K$-fold cross validation use $K = 5$.

In [ ]:
# note that this code takes a bit to run!

# as first step let's write first very explicitely the code for what we need, then we can make slicker

# we need an objective function to evaluate the values of the grid as we change the number
# of nearest neighbors
def objective_fun(Knn,K=5):
    # for a given number of nearest neighbors we first get a split of the data
    # into training and testing
    indexes_split = KFold_split(data_size, K)

    average_accuracy = 0
    # then for each split we compute the predicted labels and quantify the error
    for i in range(K):
        # this is the "leave one out" for training which is used for valdiation
        training_indexes = np.concatenate([indexes_split[j] for j in range(K) if (j != i)])

        predicted_labels = nearest_neighbour_classification(height_weight_input[indexes_split[i]],
                                                                        height_weight_input[training_indexes],
                                                                        height_weight_output[training_indexes], Knn)

        accuracy = classification_accuracy(predicted_labels, height_weight_output[indexes_split[i]])
        average_accuracy += accuracy / K
        
    error = 1.-average_accuracy

    return error


Knn=2
data_size = len(height_weight_input)
Knn = list(range(1,21,1))

grid_search(objective_fun, Knn)


In [ ]:
# we can do the same thing more concisely using the KFold_cross_validation function defined above

K=5
data_size = len(height_weight_input)
Knn = list(range(1,21,1))


evaluation = lambda testing_data_inputs,training_data_inputs, training_data_outputs, k: nearest_neighbour_classification(testing_data_inputs,training_data_inputs,training_data_outputs,no_of_neighbours=k)

missclassification_evaluation= lambda predicted_labels,true_labels: classification_accuracy(predicted_labels,true_labels)


K_objective_function = lambda k: KFold_cross_validation_knn(height_weight_input,height_weight_output, K,evaluation,missclassification_evaluation,k)

optimal_no_of_neighbours = grid_search(K_objective_function, Knn)
optimal_classification_error = K_objective_function(optimal_no_of_neighbours)

In [ ]:
print("Optimal number of neighbours one needs to consider is equal to {n}. \nIn this case the prediction error would be as little as {e:2.2f}%."\
      .format(n = optimal_no_of_neighbours, e = 100*optimal_classification_error))

Test your answers with the unit tests

In [ ]:
assert_array_almost_equal(optimal_no_of_neighbours, 15)

In [ ]:
assert_array_almost_equal(optimal_classification_error, 0.0796)

7. Visualise your results with your favourite visualisation tools, e.g. Pyplot from Matplotlib. You need to plot data points together with the decision areas

In [ ]:
grid_seed_heights = 100
grid_seed_weights = 100

# first we build the grid for the plot
grid_heights = np.linspace(np.min(height_weight_input[:, 0]),
                           np.max(height_weight_input[:, 0]),grid_seed_heights)

grid_weights = np.linspace(np.min(height_weight_input[:, 1]),
                           np.max(height_weight_input[:, 1]),grid_seed_weights)

height_grid_data, weight_grid_data = np.meshgrid(grid_heights,grid_weights,sparse=False) 

full_grid_data = np.c_[height_grid_data.reshape(-1, 1),weight_grid_data.reshape(-1, 1)]

# then we get the labels out of the best model
full_grid_labels =nearest_neighbour_classification(full_grid_data,height_weight_input,
                                                   height_weight_output,
                                                   optimal_no_of_neighbours)


# then we can plot it
plt.imshow(full_grid_labels.reshape(grid_seed_heights, grid_seed_weights),
           cmap='Spectral',origin='lower',extent=[54, 82, 50, 270],aspect="auto",alpha=0.7)

# we can also overlay the datapoints
height_weight_de_standardised = de_standardise(height_weight_input,
                                                   height_weight_row_of_means,
                                                   height_weight_row_of_stds)
plt.scatter(height_weight_de_standardised[:,0][height_weight_output == 0], \
                height_weight_de_standardised[:,1][height_weight_output == 0], s=1, c = 'Red',alpha=0.1)
plt.scatter(height_weight_de_standardised[:,0][height_weight_output == 1], \
                height_weight_de_standardised[:,1][height_weight_output == 1], s=1, c = 'Blue',alpha=0.1)

plt.show()

### Binary logistic regression
In the second part we will learn how to solve the binary classification problem for titanic data using logistic regression method. The data consists of famous Titanic ship passengers details including:
- $\mathtt{Age}$ - age of the passenger;
- $\mathtt{Embarkation\,\,\,port}$ - one of three ports (Southampton, Queenstown, Cherbourg) where the passenger has joined the trip;
- $\mathtt{Fare}$ - price the passenger paid for a ticket;
- $\mathtt{Parent/children}$ - number of parents/children of the passenger on board;
- $\mathtt{Ticket class}$ - one of three ticket classes (1,2,3);
- $\mathtt{Sex}$ - sex of the passenger;
- $\mathtt{Siblings/spouses}$ - number of siblings/spouses of the passenger on board;
- $\mathtt{Survival}$ - binary value representing whether the passenger has survived (1) or not (0).

We start by loading data. Please pay attention to how we have dealt with the embarkation port column: instead of one column with categorical data we introduce three binary columns, each representing the information on whether the passenger joined at a specific port or not. **Important:** please check that files $\mathtt{titanic.csv}$ is located in the same folder with your Jupyter notebook.

In [ ]:
titanic_dataset_path = "titanic.csv"
age = np.genfromtxt(titanic_dataset_path,
                    delimiter=",",
                    skip_header=1,
                    usecols=[0]).astype(float)
embarkation_port = np.genfromtxt(titanic_dataset_path, delimiter=",", \
                                 skip_header=1, usecols=[1], dtype = None, encoding=None)
embarkation_port_S = np.where(embarkation_port == 'S', 1, 0).astype(int)
embarkation_port_Q = np.where(embarkation_port == 'Q', 1, 0).astype(int)
embarkation_port_C = np.where(embarkation_port == 'C', 1, 0).astype(int)
fare = np.genfromtxt(titanic_dataset_path,
                     delimiter=",",
                     skip_header=1,
                     usecols=[2]).astype(float)
parent_children = np.genfromtxt(titanic_dataset_path,
                                delimiter=",",
                                skip_header=1,
                                usecols=[3]).astype(int)
ticket_class = np.genfromtxt(titanic_dataset_path,
                             delimiter=",",
                             skip_header=1,
                             usecols=[4]).astype(int)
sex = np.genfromtxt(titanic_dataset_path, delimiter=",", skip_header=1, usecols=[5],\
                    converters = {5:lambda x: 0 if x == b'male' else 1}).astype(int)
siblings_spouses = np.genfromtxt(titanic_dataset_path,
                                 delimiter=",",
                                 skip_header=1,
                                 usecols=[6]).astype(int)
survival = np.genfromtxt(titanic_dataset_path,
                         delimiter=",",
                         skip_header=1,
                         usecols=[7]).astype(int)
titanic_input = np.c_[age,fare,parent_children,ticket_class,\
                      sex,siblings_spouses,embarkation_port_S,embarkation_port_Q,embarkation_port_C]
titanic_labels = survival.reshape(-1, 1)
titanic_input, titanic_row_of_means, titanic_row_of_stds = standardise(
    titanic_input)

In [ ]:
def plot_titanic(data_input, data_output):
    characteristics_names = \
    ['Age', 'Fare', 'ParentChildren', 'Class', 'Sex', 'SiblingsSpouses', 'Southampton', 'Queenstown','Cherbourg']
    number_of_characteristics = data_input.shape[1]
    titanic_de_standardised = de_standardise(data_input, titanic_row_of_means,
                                             titanic_row_of_stds)
    titanic_dataframe = pd.DataFrame(data=titanic_de_standardised,
                                     columns=characteristics_names)
    titanic_dataframe['Survival'] = data_output
    sns.set_theme(style="ticks", color_codes=True)
    g = sns.PairGrid(titanic_dataframe,
                     vars=titanic_dataframe[:-1],
                     hue='Survival')
    g.map_offdiag(sns.scatterplot)
    g.map_diag(plt.hist)
    plt.tight_layout
    plt.show()

In [ ]:
plot_titanic(titanic_input, titanic_labels)

1. Implement function $\mathtt{linear\_regression\_data}$ that outputs the linear regression data matrix defined as
$$
\mathbf{\Phi\left(X\right)} = 
\begin{pmatrix}
1 & x^{(1)}_1 & x^{(1)}_2 & \ldots & x^{(1)}_d \\
1 & x^{(2)}_1 & x^{(2)}_2 & \ldots & x^{(2)}_d \\
\vdots & \vdots & \vdots & \ddots & \vdots & \\
1 & x^{(s)}_1 & x^{(s)}_2 & \ldots & x^{(s)}_d \\
\end{pmatrix}
$$
The function $\mathtt{linear\_regression\_data}$  should take the NumPy array *data_inputs* as argument. Here, *data_inputs* is supposed to be a data matrix containing all inputs in the matrix form as follows
$$
\mathbf{X} = 
\begin{pmatrix}
x^{(1)}_1 & x^{(1)}_2 & \ldots & x^{(1)}_d \\
x^{(2)}_1 & x^{(2)}_2 & \ldots & x^{(2)}_d \\
\vdots & \vdots & \ddots & \vdots & \\
x^{(s)}_1 & x^{(s)}_2 & \ldots & x^{(s)}_d \\
\end{pmatrix}.
$$
The function should output data matrix $\mathbf{\Phi\left(X\right)}$.

In [ ]:
def linear_regression_data(data_inputs):
    first_column = np.ones((len(data_inputs), 1))
    X_matrix = np.c_[first_column, data_inputs]
    return X_matrix

2. Implement a function $\mathtt{model\_function}$ that takes two arguments _data_matrix_ and _weights_ and outputs the values of the model function $f\left(\mathbf{x},\mathbf{w}\right) = \left\langle \phi\left(\mathbf{x}\right),\mathbf{w} \right\rangle$ evaluated for all data samples $\mathbf{x}_i$, $i=\overline{1,s}$.

In [ ]:
def model_function(data_matrix, weights):
    return data_matrix @ weights

Test your function with the following unit tests

In [ ]:
test_data_matrix = np.array([[1, -1, 2, -3], [1, 0, 2, 4], [1, -3, 0, 5]])
test_weights = np.array([[2], [0], [1], [-1]])
assert_array_almost_equal(model_function(test_data_matrix, test_weights),
                          np.array([[7], [0], [-3]]))

3. Write a function $\mathtt{logistic\_function}$ that takes an argument named *inputs* and returns the output of the $\sigma(x)$ sigmoid function, i.e.
\begin{equation*} 
\sigma(x): = \frac{1}{1+\mathrm{e}^{-x}} \, , 
\end{equation*}
applied to the *input*. Here $x$ is the mathematical notation for the argument inputs.

In [ ]:
def logistic_function(inputs):
    return 1 / (1 + np.exp(-inputs))

Test your function with the following unit tests

In [ ]:
test_inputs = np.array([[0], [np.log(25)], [-6], [np.log(9)], [2]])
assert_array_almost_equal(logistic_function(test_inputs), np.array([[1/2], [25/26], [0.0024726231566347743], \
                            [9/10], [(np.exp(2))/(1 + np.exp(2))]]))

In [ ]:
def binary_prediction_function(data_matrix, weights):
    # the binary classification can be obtained by applying the logistic function to the predicted ys i.e., Xw
    probability = logistic_function(model_function(data_matrix, weights))
    return probability > 1/2

Test your function with the following unit tests

In [ ]:
test_data_matrix = np.array([[1, -1, 2, -3], [1, 0, 2, 4], [1, -3, 0, 5]])
test_weights = np.array([[2], [0], [1], [-1]])
assert_array_almost_equal(
    binary_prediction_function(test_data_matrix, test_weights),
    np.array([[1], [0], [0]]))

5. Implement a function $\mathtt{gradient\_descent}$ that performs gradient descent to numerically approximate a minimiser of a convex function. The function should take the following arguments
- *objective* - a lambda-function representing function $E$. This itself should take a NumPy array as its argument and return a real number.
- *gradient* - a lambda-function representing function $\nabla E$. This itself should take a NumPy array as its argument and return a NumPy array representation of the gradient $\nabla E$.
- *initial_ weights* - a NumPy array with initial values $\mathbf{w}^{(0)}$ for the first iterate 
- *step_size* - a step-size parameter $\tau$ for the gradient descent step
- *no_of_iterations* - an integer parameter that controls the number of iterations
- *print_output* - an integer parameter that controls how often you are printing an intermediate result. If say *print_output = 100*, then after every 100th iteration you are asked to print your current iterate and a value of the objective as *Iteration k/m, objective = o.*, where $k$ is a number of current iteration, $m$ is a total number of iterations, and $o$ is a value of the objective evaluated at current iterate.

Implement the function so that it returns a NumPy array of the weights obtained after gradient descent together with a list of objective values for all iterates.

In [ ]:
def gradient_descent(objective,gradient,initial_weights,step_size=1,no_of_iterations=100,print_output=10):
    
    objective_values = []
    weights = np.copy(initial_weights)
    objective_values.append(objective(weights))
    
    for counter in range(no_of_iterations):
        weights -= step_size * gradient(weights)
        objective_values.append(objective(weights))
        if (counter + 1) % print_output == 0:
            print("Iteration {k}/{m}, objective = {o}.".format(k=counter+1,m=no_of_iterations, 
                                                               o=objective_values[counter]))
            
    print("Iteration completed after {k}/{m}, objective = {o}.".format(k=counter+ 1,
                                                                       m=no_of_iterations, 
                                                                       o=objective_values[counter]))
    return weights, objective_values

Test your function with the following unit tests

In [ ]:
test_matrix_m = np.array([[3, 1], [2, 4]])
test_vector_v = np.array([5, 6])
# let us consider the usual E(x) function
test_objective = lambda x: x.T @ (test_matrix_m @ x) + x @ test_vector_v
#we know the gradient
test_gradient = lambda x: (test_matrix_m + test_matrix_m.T) @ x + test_vector_v

test_initial_weights = np.array([0.0, 0.0])
test_step_size = 0.9 / (np.linalg.norm(test_matrix_m + test_matrix_m.T))
test_no_of_iterations = 100
test_print_output = 10
assert_array_almost_equal(gradient_descent(test_objective, test_gradient, \
                                           test_initial_weights,test_step_size,\
                                           test_no_of_iterations, test_print_output)[0],np.array([-0.564103, -0.538462]))

6. Write two functions that implement the cost function for binary logistic regression as well as its gradient, as defined below. 
$$
\mathrm{L}\left(\mathbf{w}\right) = \frac{1}{s} \left(
\sum\limits_{i=1}^s \log\left[1+\exp\left(f\left(\mathbf{x}^{(i)},\mathbf{w}\right)\right)\right] - y_i\cdot f\left(\mathbf{x}^{(i)},\mathbf{w}\right)
\right),
$$
where $\mathbf{x}^{(i)}$ is a vector representing $i$-th data sample and $f$ is a model function. In the case of linear model function $f\left(\mathbf{x},\mathbf{w}\right) = \left\langle \phi\left(\mathbf{x}\right),\mathbf{w} \right\rangle$ one has
$$
\nabla \mathrm{L}\left(\mathbf{w}\right) = \frac{1}{s} \left(
\sum\limits_{i=1}^s \phi\left(\mathbf{x}^{(i)}\right)\cdot\sigma
\left(\left\langle \phi\left(\mathbf{x}^{(i)}\right),\mathbf{w} \right\rangle \right) - y_i\cdot \phi\left(\mathbf{x}^{(i)}\right)
\right),
$$
with $\phi\left(\mathbf{x}^{(i)}\right)$ being an augmented $i$-th data vector containing additional coordinate $1$.

The function for the cost function is named $\mathtt{binary\_logistic\_regression\_cost\_function}$ and should take the NumPy arrays _data_matrix_, _data_labels_ and _weights_ as arguments. Subsequently, write a method $\mathtt{binary\_logistic\_regression\_gradient}$ that takes the same inputs as $\mathtt{binary\_logistic\_regression\_cost\_function}$ and computes the gradient of the binary logistic regression cost function as defined in the lecture.

In [ ]:
def binary_logistic_regression_cost_function(data_matrix, data_labels,weights):
    regression_outputs = model_function(data_matrix, weights)
    return np.mean(np.log(1 + np.exp(regression_outputs)) -data_labels * regression_outputs)

In [ ]:
def binary_logistic_regression_gradient(data_matrix, data_labels, weights):
    return data_matrix.T @ (logistic_function(model_function(data_matrix, weights)) - data_labels) / len(data_matrix)

Test your function with the following unit tests

In [ ]:
test_data_matrix = np.array([[1, -1, 2, -3], [1, 0, 2, 4], [1, -3, 0, 5]])
test_data_labels = np.array([[1], [0], [0]])
test_weights = np.array([[2], [0], [1], [-1]])
assert_array_almost_equal(binary_logistic_regression_cost_function(test_data_matrix,
                                             test_data_labels, test_weights),0.24754867)

In [ ]:
test_data_matrix = np.array([[1, -1, 2, -3], [1, 0, 2, 4], [1, -3, 0, 5]])
test_data_labels = np.array([[1], [0], [0]])
test_weights = np.array([[2], [0], [1], [-1]])
assert_array_almost_equal(binary_logistic_regression_gradient(test_data_matrix,test_data_labels,test_weights), \
                          np.array([[0.182172],[-0.047122],[0.332726], [0.746621]]))

7. Finally use all the above methods to evaluate predicted class labels (survival) for all Titanic passengers. Use all data to train your model and evaluate classification corresponding classification accuracy. You are expected to:
- build a linear regression data matrix _titanic_data_matrix_ based on the _titanic_input_ data
- evaluate optimal weights by running the gradient descent method: take initial weights to be zero and run $N = 2000$ iterations with a step-size parameter equal to $\tau = 3.9\cdot s \cdot\left\|\mathbf{\Phi\left(X\right)}\right\|^{-2}$, where $s$ is a number of data samples, $\mathbf{\Phi\left(X\right)}$ is a mathematical representation of _data_matrix_. Assign a variable _titanic_optimal_weights_ with your result.
- evaluate predicted class labels for all data samples. Store the result in _titanic_predicted_labels_ variable.
- evaluate accuracy rate of your prediction (should be a value between 0 and 1) and store its value in a variable _titanic_accuracy_rate_

In [ ]:
# first let's get the data matrix from the inputs
titanic_data_matrix = linear_regression_data(titanic_input)

# we then define the cost function
titanic_cost_function = lambda weights: binary_logistic_regression_cost_function(titanic_data_matrix,
                                                                                 titanic_labels, weights)
# then we get the gradient
titanic_gradient_function = lambda weights: binary_logistic_regression_gradient(titanic_data_matrix,
                                                                                titanic_labels, weights)

# start with w^0
titanic_initial_weights = np.zeros((len(titanic_data_matrix.T), 1))

# set the step size
titanic_step_size = 3.9 * len(titanic_data_matrix) / (np.linalg.norm(titanic_data_matrix))**2

# apply gradient descent
titanic_optimal_weights, titanic_objective_values_ =gradient_descent(titanic_cost_function,
                                                                     titanic_gradient_function,
                                                                     titanic_initial_weights,
                                                                     titanic_step_size,2000, 200)

# get the predicte labels
predicted_labels= binary_prediction_function(titanic_data_matrix,titanic_optimal_weights)

# measure the accuracy comparing predicted VS real labels
titanic_accuracy_rate = classification_accuracy(predicted_labels,titanic_labels)

Test your function with the following unit tests

In [ ]:
assert_array_almost_equal(titanic_optimal_weights, np.array([[-0.65370587], [-0.55872717],\
                                                             [0.09439554], [-0.0750632],  [-0.94173469],\
                                                             [1.29812358], [-0.38153153], [-0.08951093], \
                                                             [0.01944579],  [0.08813609]]))

In [ ]:
assert_array_almost_equal(titanic_accuracy_rate, 0.8024691358024691)

In [ ]:
print("The optimal weights are w = {w}.T.".format(w=titanic_optimal_weights.T))
print("The classification accuracy for the training set is {p} %.".format(
    p=100 * titanic_accuracy_rate))

In [ ]:
# let's perform a kfold validation of the model first
# we need to modify the function of gradient descent in order to pass both x and y


def gradient_descent_2(objective,gradient,initial_weights,data_matrix, labels,step_size=1,no_of_iterations=100,print_output=10):
    
    objective_values = []
    weights = np.copy(initial_weights)
    objective_values.append(objective(weights,data_matrix, labels))
    
    for counter in range(no_of_iterations):
        weights -= step_size * gradient(weights,data_matrix, labels)
        objective_values.append(objective(weights,data_matrix, labels))
    return weights, objective_values


def kfold_logistic(input_data,labels,K,cost_function,gradient_function):

    data_size = len(input_data)
    indexes_split = KFold_split(data_size, K)

    average_accuracy = 0
    
    data_matrix = linear_regression_data(input_data)
    optimal_weights_k_fold=np.zeros((len(data_matrix.T), 1))
    
    
    for i in range(K):
        training_indexes = np.concatenate([indexes_split[j] for j in range(K) if (j != i)])

        data_matrix = linear_regression_data(input_data[training_indexes])

        # start with w^0
        initial_weights = np.zeros((len(data_matrix.T), 1))

        # set the step size
        step_size = 3.9 * len(data_matrix) / (np.linalg.norm(data_matrix))**2

        # apply gradient descent
        optimal_weights, objective_values_ =gradient_descent_2(cost_function,gradient_function,
                                                            initial_weights,data_matrix,
                                                               labels[training_indexes],
                                                                step_size,2000, 200)

        # get the predicte labels
        data_matrix_test = linear_regression_data(input_data[indexes_split[i]])

        predicted_labels= binary_prediction_function(data_matrix_test,optimal_weights)

        # measure the accuracy comparing predicted VS real labels
        accuracy_rate = classification_accuracy(predicted_labels,labels[indexes_split[i]])
        average_accuracy+=accuracy_rate/K

        optimal_weights_k_fold += optimal_weights / K

    return optimal_weights_k_fold,average_accuracy

# we need to change also the cost and gradient functions as we need to evaluate them in a subset of data

cost_function = lambda weights,data_matrix,labels: binary_logistic_regression_cost_function(data_matrix,labels, weights)
gradient_function = lambda weights,data_matrix,labels: binary_logistic_regression_gradient(data_matrix,labels, weights)


In [ ]:
K=5
optimal_weights_k_fold,average_accuracy=kfold_logistic(titanic_input,titanic_labels,K,cost_function,gradient_function)
    
print (average_accuracy)
print (optimal_weights_k_fold)

## What about understanding a bit more the performance of the model?

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
titanic_data_matrix = linear_regression_data(titanic_input)

predicted_labels= binary_prediction_function(titanic_data_matrix,optimal_weights_k_fold)


print(confusion_matrix(titanic_labels, predicted_labels))

In [ ]:
conf_matrix=confusion_matrix(titanic_labels, predicted_labels,normalize='all')
sns.heatmap(conf_matrix, annot=True)
plt.show()

In [ ]:
def binary_prediction_function_threshold(data_matrix, weights,threshold):
    # the binary classification can be obtained by applying the logistic function to the predicted ys i.e., Xw
    probability = logistic_function(model_function(data_matrix, weights))
    return probability > threshold

In [ ]:
titanic_data_matrix = linear_regression_data(titanic_input)

threshold=0.
delta_t=0.005
values=[]
while threshold<=1:
    
    predicted_labels= binary_prediction_function_threshold(titanic_data_matrix,optimal_weights_k_fold,threshold)
    tn, fp, fn, tp = confusion_matrix(titanic_labels, predicted_labels).ravel()
    TPR=tp/(tp+fn)
    FPR=fp/(tn+fp)
    values.append([FPR,TPR,threshold])
    threshold+=delta_t

In [ ]:
x_val=[]
y_val=[]
for i in values:
    x_val.append(i[0])
    y_val.append(i[1])

plt.plot(x_val,y_val)
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.show()

### The ROC curve should be evaluated out of sample!

In [ ]:
## so we can compute the ROC for each fold!


def get_ROC(data_matrix,weights,delta_t,labels):
    threshold=0.
    values=[]
    while threshold<=1:

        predicted_labels= binary_prediction_function_threshold(data_matrix,weights,threshold)
        tn, fp, fn, tp = confusion_matrix(labels, predicted_labels).ravel()
        TPR=tp/(tp+fn)
        FPR=fp/(tn+fp)
        values.append([FPR,TPR,threshold])
        threshold+=delta_t

    return values


def kfold_logistic_with_ROC(input_data,labels,K,cost_function,gradient_function):

    data_size = len(input_data)
    indexes_split = KFold_split(data_size, K)

    average_accuracy = 0
    values_ROC={}
    for i in range(K):
        training_indexes = np.concatenate([indexes_split[j] for j in range(K) if (j != i)])

        data_matrix = linear_regression_data(input_data[training_indexes])

        # start with w^0
        initial_weights = np.zeros((len(data_matrix.T), 1))

        # set the step size
        step_size = 3.9 * len(data_matrix) / (np.linalg.norm(data_matrix))**2

        # apply gradient descent
        optimal_weights, objective_values_ =gradient_descent_2(cost_function,gradient_function,
                                                            initial_weights,data_matrix,
                                                               labels[training_indexes],
                                                                step_size,2000, 200)

        # get the predicte labels
        data_matrix_test = linear_regression_data(input_data[indexes_split[i]])

        predicted_labels= binary_prediction_function(data_matrix_test,optimal_weights)

        # measure the accuracy comparing predicted VS real labels
        accuracy_rate = classification_accuracy(predicted_labels,labels[indexes_split[i]])
        average_accuracy+=accuracy_rate/K

        optimal_weights_k_fold = optimal_weights / K

        
        values_ROC[i]=get_ROC(data_matrix_test,optimal_weights,0.005,labels[indexes_split[i]])
        
        

    return optimal_weights_k_fold,average_accuracy,values_ROC

In [ ]:
K=5
optimal_weights_k_fold,average_accuracy,values_ROC=kfold_logistic_with_ROC(titanic_input,titanic_labels,K,cost_function,gradient_function)
    
print (average_accuracy)
print (optimal_weights_k_fold)


In [ ]:

for i in values_ROC:
    x_val=[]
    y_val=[]
    for j in values_ROC[i]:
        x_val.append(j[0])
        y_val.append(j[1])

    plt.plot(x_val,y_val,label='fold'+str(i+1))


    
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc=[1.05,0.4])
plt.show()